In [ ]:
!pip install flask-ngrok
!pip install Flask==3.0.0 pyngrok==7.1.2
%pip install --quiet --upgrade langchain langchain-community langchain-pinecone
%pip install -U --quiet tiktoken langchainhub langgraph langchain-text-splitters
%pip install -qU langchain-openai
!pip install --upgrade --upgrade-strategy eager "regex" "charset-normalizer<4" "idna" "urllib3<3" "certifi" "requests" "anyio<5" "distro<2" "sniffio" "h11<0.15" "httpcore==1.*"  "annotated-types" "typing-extensions<5" "pydantic-core==2.27.1" "pydantic<3" "jiter<1" "tqdm" "colorama" "openai" "tiktoken" "httpx<0.28"

In [ ]:
port = 5000

In [ ]:
from pyngrok import ngrok

In [ ]:
from google.colab import userdata
ngrok_token = userdata.get('NGROK_TOKEN')

In [ ]:
ngrok.set_auth_token(ngrok_token)
ngrok.connect(port).public_url

'https://7549-34-134-86-113.ngrok-free.app'

In [ ]:
import getpass
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from flask import Flask, request, jsonify
import requests
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from pinecone import Pinecone, ServerlessSpec
from langchain_core.messages import BaseMessage, HumanMessage
from google.colab import userdata

In [ ]:
##Langchain configuration
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_PROJECT="pr-gargantuan-whelp-32"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from flask import Flask, request, jsonify
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import json
import time

app = Flask(__name__)

PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = userdata.get("PINECONE_INDEX_NAME")

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embeddings are 1536 dimensions
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

# Replace VertexAI with OpenAI components
embedding_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectorstore = PineconeVectorStore(index=index, embedding=embedding_model)
retriever = vectorstore.as_retriever()

# Initialize OpenAI LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",  # or gpt-3.5-turbo
    temperature=0,
    max_retries=6,
    api_key=OPENAI_API_KEY,
)



In [ ]:
@app.route('/api2', methods=['POST'])
def handle_request12():
    try:
        # Parse the JSON request body
        data = request.get_json()
        print(data)
        question = data.get('question')
        context = data.get('data')
        if not question or not context:
            return jsonify({'error': 'Invalid input data'}), 400
        print(question)
        context = add_context(context)
        genai.configure(api_key=GEMINI_API_KEY)
         model = genai.GenerativeModel('gemini-1.5-flash')
        # Send the question and context to the Gemini API
        prompt = f"Context: {context}\nGiven the above context,  extract information from the context relevant to the question. Use the extracted information to answer the question. Justify your answer.\n Question: {question}\n"
        createContextEmbeddings(context)
        rag_response = call_rag_model(prompt)
        print(rag_response)
        return jsonify({'answer': rag_response}), 200
        #return jsonify({'answer': gemini_response.text.strip()}), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500



#         raise Exception(f'Gemini API error: {response.status_code} {response.text}')

app.run(port=port)

In [ ]:
def add_context(context):
  context['user_profile']['description of fields'] = json.dumps({
                "height_cm": "The height of the user in centimeters. Used for calculating metrics like BMI and other fitness indicators.",
                "weight_kg": "The weight of the user in kilograms. Helps determine caloric needs and progress toward health goals.",
                "target_steps": "The daily step count goal set by the user or system to encourage physical activity.",
                "target_calories": "The daily target of calories to be burned through activity, critical for weight management goals."
            }, indent = 2)
  context['health_metrics']['description of fields'] = json.dumps({
                "heart_rate_bpm": "User's heart rate in beats per minute, an indicator of cardiovascular health and exercise intensity.",
                "body_temperature_celsius": "The user's body temperature in Celsius, important for detecting illness or physiological changes.",
                "steps_count": "The number of steps taken by the user during the current period. Tracks physical activity levels.",
                "calories_burned": "The number of calories burned through activity and metabolic functions, reflecting energy expenditure.",
                "blood_oxygen_percentage": "The oxygen saturation in the user's blood, a measure of respiratory and circulatory efficiency.",
                "glucose_mg_dl": "Blood glucose levels in milligrams per deciliter, a key metric for metabolic health.",
                "heart_rate_variability_ms": "Variability in the time between heartbeats, measured in milliseconds, indicating stress and recovery levels.",
                "sleep_stages": "Sequence of sleep stages (e.g., Awake, REM, Deep) recorded during rest. Useful for sleep quality analysis.",
                "blood_pressure": "The user's blood pressure in mmHg (systolic/diastolic), important for cardiovascular health monitoring."
            }, indent = 2)
  context['weather_data']['description of fields'] = json.dumps({
            "temperature_celsius": "The current temperature at the user's location in Celsius. Helps in activity planning and weather adaptation.",
            "humidity_percentage": "The percentage of moisture in the air. Higher levels can affect comfort and exertion levels.",
            "precipitation_mm": "The amount of precipitation in millimeters. Important for outdoor activity planning.",
            "snowfall_mm": "The amount of snowfall in millimeters. Relevant for users in cold climates or outdoor winter sports.",
            "air_quality_index": "An index representing air quality. Lower values signify healthier air for breathing.",
            "air_quality_description": "Descriptive indicator of air quality (e.g., 'Good', 'Moderate'). Helps assess environmental conditions for activities.",
            "uv_index": "The ultraviolet index at the location. Higher values indicate a greater need for sun protection.",
            "wind_speed_ms": "The speed of wind in meters per second. Important for outdoor activities and comfort.",
            "wind_gust_ms": "The maximum speed of wind gusts in meters per second. Helps anticipate extreme conditions.",
            "pressure_hpa": "The atmospheric pressure in hectopascals. Sudden changes can indicate weather shifts."
        }, indent = 2)
  context['sensor_data']['description of fields'] = json.dumps( {
            "TMD3719 Ambient Light": "Measurement of ambient light intensity. Used for determining optimal brightness settings or light exposure levels.",
            "Gravity Sensor": "Data on gravitational force along different axes. Useful for detecting orientation or movement patterns.",
            "LSM6DSV Gyroscope-Uncalibrated": "Measures rotational motion in degrees per second. Useful for activity recognition or device orientation tracking.",
            "Step Counter": "Tracks the total number of steps detected by the device. Core metric for activity monitoring.",
            "Device Orientation": "Provides the current orientation of the device. Often used for navigation or activity context.",
            "MMC56X3X Magnetometer": "Measures magnetic field strength. Used in navigation and determining magnetic direction.",
            "LSM6DSV Accelerometer-Uncalibrated": "Measures acceleration along different axes. Useful for detecting movement or analyzing activity.",
            "Orientation Sensor": "Tracks the device's pitch, roll, and yaw. Helpful for applications requiring precise motion tracking.",
            "TMD3719 Proximity (wake-up)": "Detects proximity to nearby objects. Often used to trigger actions like wake-up events.",
            "Linear Acceleration Sensor": "Measures linear acceleration excluding gravity. Useful for analyzing specific movements like jumps or runs.",
            "LSM6DSV Gyroscope": "Measures angular velocity. Useful for analyzing rotation or angular motion.",
            "Rotation Vector Sensor": "Combines accelerometer and gyroscope data to calculate device orientation in 3D space.",
            "Game Rotation Vector Sensor": "Optimized for low latency, provides rotation data for gaming or real-time applications.",
            "MMC56X3X Magnetometer-Uncalibrated": "Uncalibrated magnetic field data for advanced navigation or environmental sensing.",
            "Auto Brightness": "Tracks changes in ambient brightness for dynamic screen adjustments.",
            "Geomagnetic Rotation Vector Sensor": "Combines magnetic field data for orientation in magnetic environments.",
            "Binned Brightness (wake-up)": "Histogram of brightness levels detected over time. Useful for light exposure analysis.",
            "LSM6DSV Temperature": "Temperature of the gyroscope sensor. Helps in device calibration and thermal monitoring.",
            "LSM6DSV Accelerometer": "Measures device acceleration. Useful for tracking movement dynamics.",
            "VD6282 Rear Light Sensor": "Monitors light intensity on the device's rear side. Used for light adaptation or monitoring.",
            "ICP20100 Pressure Sensor": "Measures atmospheric pressure. Relevant for weather monitoring and altitude detection.",
            "ICP20100 Temperature": "Temperature data from the pressure sensor. Useful for environmental sensing.",
            "Tilt Sensor (wake-up)": "Detects tilts to trigger actions like waking up the device or user."
        }, indent = 2)
  context['location']['description of fields'] = json.dumps({
            "latitude": "The user's current latitude in decimal degrees. Used for location-based services and context.",
            "longitude": "The user's current longitude in decimal degrees. Combined with latitude to determine precise location."
        }, indent = 2)
  context['fitbit_data']['description of fields']  = json.dumps({
        "activities": {
            "description": "A log of activities recorded by the Fitbit device."
        },
        "summary": {
            "caloriesOut": {
                "description": "Total calories burned during the day."
            },
            "activityCalories": {
                "description": "Calories burned specifically through activities."
            },
            "caloriesBMR": {
                "description": "Calories burned through basic metabolic functions."
            },
            "activeScore": {
                "description": "A Fitbit-provided score for daily activity."
            },
            "steps": {
                "description": "Steps recorded by Fitbit during a tracked period."
            },
            "floors": {
                "description": "Number of floors climbed during the day."
            },
            "elevation": {
                "description": "Elevation gain during activities."
            },
            "sedentaryMinutes": {
                "description": "Minutes spent in sedentary behavior."
            },
            "lightlyActiveMinutes": {
                "description": "Minutes spent in light physical activity."
            },
            "fairlyActiveMinutes": {
                "description": "Minutes spent in moderate physical activity."
            },
            "veryActiveMinutes": {
                "description": "Minutes spent in vigorous physical activity."
            },
            "distances": {
                "description": "Distances traveled during different activity types."
            },
            "marginalCalories": {
                "description": "Additional calories burned through activities."
            },
            "restingHeartRate": {
                "description": "The user's resting heart rate."
            },
            "heartRateZones": {
                "description": "Heart rate zones recorded during the day."
            }
        },
        "goals": {
            "caloriesOut": {
                "description": "The target number of calories to burn daily."
            },
            "steps": {
                "description": "The target number of steps to achieve daily."
            },
            "distance": {
                "description": "The target distance to travel in kilometers."
            },
            "floors": {
                "description": "The target number of floors to climb daily."
            },
            "activeMinutes": {
                "description": "The target number of active minutes daily."
            }
        }
    })
  return context

In [ ]:
def call_rag_model(question):
  retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
  prompt = PromptTemplate(
        template="""
        Here is the user context data: \n\n {context} \n\n
        Here is the user question: {question} \n
        Given the above context,  extract information from the context relevant to the question. Use the extracted information to answer the question. Justify your answer.\n """,
        input_variables=["context", "question"],
    )
  rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
  )

  response = rag_chain.invoke(question)
  return response

In [ ]:
##Creating the vector database of historical context and historical conversation embeddings
# save the context with date and time of sensor data to allow it to predict user habits
def storeEmbeddings(embedding, context):
  embedding_id = str(uuid.uuid4())
  index = pc.Index(index_name)
  index.upsert([(embedding_id, embedding, context)])


In [ ]:
###Getting the historical Data and creating embedding
import pandas as pd
import uuid
def createContextEmbeddings(data):
  formatted_data = f"""
  User Profile: Height: {data['user_profile']['height_cm']} cm, Weight: {data['user_profile']['weight_kg']} kg, Target Steps: {data['user_profile']['target_steps']}, Target Calories: {data['user_profile']['target_calories']}
  User Profile description: {json.dumps(data['user_profile']['description of fields'])}
  Health Metrics: {json.dumps(data['health_metrics'], indent=2)}
  Weather Data: {json.dumps(data['weather_data'], indent=2)}
  Sensor Data: {json.dumps(data['sensor_data'], indent=2)}
  Fitbit data: {json.dumps(data['fitbit_data'], indent=2)}
  Location: Latitude: {data['location']['latitude']}, Longitude: {data['location']['longitude']}
  """
  formatted_dict = {'text':json.dumps({
  'User Profile':  json.dumps(data['user_profile'], indent=2),
  'Health Metrics': json.dumps(data['health_metrics'], indent=2),
  'Weather Data': json.dumps(data['weather_data'], indent=2),
  'Sensor Data': json.dumps(data['sensor_data'], indent=2),
  'Location': json.dumps(data['location'], indent=2),
  'Fitbit data': json.dumps(data['fitbit_data'], indent=2)})}
  embedding = embedding_model.embed_query(formatted_data)
  storeEmbeddings(embedding, formatted_dict)
  return "Finished Storing the embedding"

In [ ]:
from langchain.tools.retriever import create_retriever_tool
#Reference: https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/#nodes-and-edges
#Reference: https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/#create-index
retriever_tool = create_retriever_tool(
    retriever,
    "user_data",
    "This tool MUST be used. It retrieves relevant documents from the database.",
)

tools = [retriever_tool]
print("Tools List:", tools)
tool_output = retriever_tool.func(prompt)
print("Retriever tool output:", tool_output)

In [ ]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from typing import Literal
from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from pydantic import BaseModel, Field

def grade_documents(state) -> Literal["generate", "rewrite"]:
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (messages): The current state

    Returns:
        str: A decision for whether the documents are relevant or not
    """

    print("---CHECK RELEVANCE---")

    # Data model
    class grade(BaseModel):
        """Binary score for relevance check."""

        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

    # LLM
    model = llm

    # LLM with tool and validation
    llm_with_tool = model.with_structured_output(grade)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.""",
        input_variables=["context", "question"],
    )

    # Chain
    chain = prompt | llm_with_tool

    messages = state["messages"]
    last_message = messages[-1]

    question = messages[0].content
    docs = last_message.content
    print(docs)
    scored_result = chain.invoke({"question": question, "context": docs})

    score = scored_result.binary_score

    if score == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return "generate"

    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        print(score)
        return "rewrite"

In [ ]:
def agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    model = llm
    model = model.bind_tools([retriever_tool])
    print("Tools bound to model:", model)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
def rewrite(state):
    """
    Transform the query to produce a better question.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with re-phrased question
    """

    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    question = messages[0].content

    msg = [
        HumanMessage(
            content=f""" \n
    Look at the input and try to reason about the underlying semantic intent / meaning. \n
    Here is the initial question:
    \n ------- \n
    {question}
    \n ------- \n
    Formulate an improved question: """,
        )
    ]

    # Grader
    model = llm
    response = model.invoke(msg)
    return {"messages": [response]}

In [ ]:
def generate(state):
    """
    Generate answer

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    print("---GENERATE---")
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]

    docs = last_message.content

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    model = llm

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | model | StrOutputParser()

    # Run
    response = rag_chain.invoke({"context": docs, "question": question})
    return {"messages": [response]}


In [ ]:
def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # Re-writing the question
workflow.add_node(
    "generate", generate
)  # Generating a response after we know the documents are relevant
# Call agent node to decide to retrieve or not
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)
workflow.add_conditional_edges(
    "agent",
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
graph = workflow.compile()

In [ ]:
import pprint
input = f"Context: {context}\nGiven the above context,  extract information from the context and retrieve documents relevant to the question. Use the extracted information and retrieved documents to answer the question. Justify your answer.\n Question: {question}\n"

inputs = {
    "messages": [
        ("user", input),
    ]
}
print(inputs)
for output in graph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

In [ ]:
##Multiquery Retriever
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.output_parsers import OutputFixingParser

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=new_parser)
multiqueryretriever = MultiQueryRetriever(
    retriever=retriever, llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output
print(prompt)
unique_docs = multiqueryretriever.invoke(input=question)


In [ ]:
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)
def search_with_tavily(query, api_key, max_results=5):
    try:
        # Initialize the client
        client = TavilyClient(api_key=userdata.get("TAVILY_API_KEY"))

        # Perform the search with parameters
        response = client.search(
            query=query,
            max_results=max_results,
            include_images=True,
            search_depth="advanced"
        )

        # Print results
        print(f"\nSearch Results for: {query}\n")
        for result in response['results'][:1]:
            print(f"Title: {result['title']}")
            print(f"Content: {result['content']}")
            print("-" * 50)

    except Exception as e:
        print(f"Error occurred: {str(e)}")